In [3]:
import os
import sys
import json
import torch
import numpy as np
import torch.nn as nn
from tqdm import tqdm
from neuralop import LpLoss
from neuralop import Trainer
from neuralop.models import SFNO
from neuralop.training import AdamW
from utils import SphericalNODataset
from torch.utils.data import DataLoader


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [10]:
exp_path = '/Users/reza/Career/DMLab/SURROGATE/results/laplace/sfno/exp_18'
cfg_path = os.path.join(exp_path, 'cfg.json')
state_path = os.path.join(exp_path, 'ckpt', 'best_model_state_dict.pt')

In [11]:
with open(cfg_path, "r", encoding='utf-8') as f:
        cfg = json.load(f)

In [ ]:
sim_paths = cfg["val_files"]
sim_paths = list(map(lambda x: x.replace('/home/rmansouri1/sun-sim/data/psi_web/6-feb-2025', ''), sim_paths))
val_dataset = SphericalNODataset(
    sim_paths,
    rho_min=cfg["rho_min"],
    rho_max=cfg["rho_max"],
)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

  0%|          | 0/227 [00:00<?, ?it/s]


HDF4Error: SD: no such file

In [ ]:
model = SFNO(
    n_modes=(cfg["n_modes"], cfg["n_modes"]),
    in_channels=1,
    out_channels=140,
    hidden_channels=cfg["hidden_channels"],
    projection_channel_ratio=2,
    factorization="dense",
)
model = model.to(device)
state = torch.load(state_path, weights_only=False, map_location=device)
model.load_state_dict(state)
l2loss = LpLoss(d=2, p=2, reduction="sum")

losses = []

model.eval()
for batch in tqdm(val_loader):
    sample = {k: v.to(device) for k, v in batch.items() if torch.is_tensor(v)}
    with torch.no_grad():
        out = model(**sample)
    loss = l2loss(out, sample["y"])
    losses.append(loss.item())

losses = np.mean(losses)

print(losses)